### Курсовой проект

**Основное**
- Дедлайн - 21 июня 23:59
- Целевая метрика precision@5
- Бейзлайн решения - [MainRecommender](https://github.com/geangohn/recsys-tutorial/blob/master/src/recommenders.py)
- Сдаем ссылку на github с решением. На github должен быть файл recommendations.csv (user_id | [rec_1, rec_2, ...] с рекомендациями. rec_i - реальные id item-ов (из retail_train.csv)

----

**Установка и загрузка**

In [13]:
import pandas as pd
import numpy as np

from scipy.sparse import csr_matrix, coo_matrix

from implicit import als

from lightgbm import LGBMClassifier

from src.metrics import precision_at_k, recall_at_k, money_precision_at_k
from src.utils import pre_filter_items, get_users_features, get_items_features, get_recommendation_5
from src.recommenders import MainRecommender

import warnings
warnings.filterwarnings("ignore")

from tqdm import tqdm
tqdm.pandas()

In [14]:
path = 'C:\\Disk\\GB\\Rec_sys\\Project\\data\\'
path_data = path + 'retail_train.csv'
path_features = path + 'product.csv'
path_user = path + 'hh_demographic.csv'

data = pd.read_csv(path_data)
item_features = pd.read_csv(path_features)
user_features = pd.read_csv(path_user)

item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)

In [15]:
test_path = path + 'retail_test1.csv'
test_data = pd.read_csv(test_path)

In [16]:
data.shape, test_data.shape

((2396804, 12), (88734, 12))

In [17]:
prices = data.groupby(['item_id'])['sales_value'].mean().reset_index()
sales_qty = data.groupby(['item_id'])['quantity'].mean().reset_index()
prices = prices.merge(sales_qty, on='item_id', how='left')
prices['price'] = [prices.iloc[i]['sales_value'] / prices.iloc[i]['quantity']\
                   if prices.iloc[i]['quantity'] > 0 else 0 for i in prices['item_id'].index]
prices.drop(columns=['sales_value', 'quantity'], axis=1, inplace=True)

In [18]:
data = data.merge(prices, on='item_id', how='left')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc,price
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0,2.385178
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0,0.945892


Расчет цены делается не особо быстро - оставил возможность загруки готовых данных на будущее.

In [21]:
val_lvl_1_size_weeks = 6
val_lvl_2_size_weeks = 3

data_train_lvl_1 = data[data['week_no'] < data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)]
data_val_lvl_1 = data[(data['week_no'] >= data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)) &
                      (data['week_no'] <= data['week_no'].max() - (val_lvl_2_size_weeks))]

data_train_lvl_2 = data_val_lvl_1.copy()  
data_val_lvl_2 = data[data['week_no'] > data['week_no'].max() - val_lvl_2_size_weeks]

In [22]:
data_train_lvl_1.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc,price
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0,2.385178
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0,0.945892


In [23]:
n_items_before = data_train_lvl_1['item_id'].nunique()
data_train_lvl_1 = pre_filter_items(data_train_lvl_1, item_features=item_features, take_n_popular=5000)
n_items_after = data_train_lvl_1['item_id'].nunique()
print(f'Decreased # items from {n_items_before} to {n_items_after}')

Decreased # items from 83685 to 5001


In [26]:
items_sold = data_train_lvl_1['item_id'].unique().tolist()
print(f'Отобрано категорий - {item_features[item_features["item_id"].isin(items_sold)]["sub_commodity_desc"].nunique()}')

Отобрано категорий - 740


In [27]:
# модель первого уровня
recommender = MainRecommender(data_train_lvl_1, weighting=True)

In [28]:
items_embeddings = recommender.items_emb_df
users_embeddings = recommender.users_emb_df

**Рекомендации первого уровня**

In [29]:
result_lvl_1 = data_val_lvl_1.groupby('user_id')['item_id'].unique().reset_index()
result_lvl_1.columns=['user_id', 'actual']
result_lvl_1.head(2)

,user_id,actual
0,1,"[853529, 865456, 867607, 872137, 874905, 87524..."
1,2,"[15830248, 838136, 839656, 861272, 866211, 870..."


In [30]:
N = 200
users = set(data_train_lvl_1['user_id'].unique().tolist())
top_popular_items = recommender.overall_top_purchases[:N]

In [31]:
result_lvl_1['own_recommendations'] = result_lvl_1['user_id'].progress_apply(lambda x: \
                                                                             recommender.get_own_recommendations(x, N=N))

100%|██████████████████████████████████████████████████████████████████████████████| 2197/2197 [00:37<00:00, 58.96it/s]


In [32]:
result_lvl_1.tail(2)

,user_id,actual,own_recommendations
2195,2499,"[861282, 921744, 1050968, 13842089, 828837, 86...","[1082185, 1029743, 995242, 1106523, 981760, 11..."
2196,2500,"[856455, 902192, 903476, 931672, 936634, 95170...","[1082185, 1029743, 995242, 1106523, 981760, 11..."


In [35]:
# Отбор для 2 уровня
users_lvl_2 = pd.DataFrame(data_train_lvl_2['user_id'].unique())
users_lvl_2.columns = ['user_id']

train_users = data_train_lvl_1['user_id'].unique()
users_lvl_2 = users_lvl_2[users_lvl_2['user_id'].isin(train_users)]

In [36]:
# рекомендации 1 уровня
users_lvl_2 = users_lvl_2.merge(result_lvl_1, on='user_id', how='left')
users_lvl_2.tail(2)

,user_id,actual,own_recommendations
2193,903,"[923746, 1005274, 1070820, 6961519]","[1082185, 1029743, 995242, 1106523, 981760, 11..."
2194,1276,"[834484, 855672, 860776, 879528, 954355, 95802...","[1082185, 1029743, 995242, 1106523, 981760, 11..."


**Фичи второго уровня**

In [37]:
user_features_upd = get_users_features(user_features, data_train_lvl_2, users_embeddings)
user_features_upd.head(2)

,user_id,marital_status_code,homeowner_desc,hh_comp_desc,age_desc_int,income_desc_int,household_size_desc_int,kid_category_desc_int,days_from_last_purchase,av_check,...,10,11,12,13,14,15,16,17,18,19
0,1,A,Homeowner,2 Adults No Kids,70,42,2,0,3,51.265000,...,6.527886,-12.539451,5.984131,-1.037484,4.962157,2.861279,0.998516,5.748231,8.489971,2.292151
1,7,A,Homeowner,2 Adults No Kids,50,61,2,0,4,41.318571,...,-0.738088,2.945041,2.845266,3.185545,0.621381,-6.108757,-5.112888,0.128096,1.424547,9.206670


 **Товары**

In [38]:
item_features_upd = get_items_features(item_features, data_train_lvl_2, items_embeddings)
item_features_upd.head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product,days_from_last_purchase_item,av_item_num_per_basket,item_value,...,10,11,12,13,14,15,16,17,18,19
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB,345.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,,345.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
item_features_upd = item_features_upd.merge(prices, on='item_id', how='left')

In [40]:
s = users_lvl_2.apply(lambda x: pd.Series(x['own_recommendations']), axis=1).stack().reset_index(level=1, drop=True)
s.name = 'item_id'
users_lvl_2 = users_lvl_2.drop('own_recommendations', axis=1).join(s)
users_lvl_2 = users_lvl_2.drop('actual', axis=1)
users_lvl_2['drop'] = 1

In [41]:
data_train_lvl_2.shape

(202837, 13)

In [42]:
targets_lvl_2 = data_train_lvl_2[['user_id','item_id']].copy()
targets_lvl_2['target'] = 1
targets_lvl_2 = users_lvl_2.merge(targets_lvl_2, on=['user_id', 'item_id'], how='left')
targets_lvl_2['target'].fillna(0, inplace= True)
targets_lvl_2.drop('drop', axis=1, inplace=True)

In [43]:
targets_lvl_2.head(2)

,user_id,item_id,target
0,2070,1082185,1.0
1,2070,1029743,0.0


In [44]:
targets_lvl_2 = targets_lvl_2.merge(user_features_upd, on='user_id', how='left')
targets_lvl_2 = targets_lvl_2.merge(item_features_upd, on='item_id', how='left')

In [45]:
targets_lvl_2.shape

(449918, 64)

In [46]:
targets_lvl_2.head(2)

,user_id,item_id,target,marital_status_code,homeowner_desc,hh_comp_desc,age_desc_int,income_desc_int,household_size_desc_int,kid_category_desc_int,...,11_y,12_y,13_y,14_y,15_y,16_y,17_y,18_y,19_y,price
0,2070,1082185,1.0,U,Unknown,Unknown,50.0,61.0,1.0,0.0,...,-0.000513,0.022309,0.029369,0.017117,-0.010773,0.016888,0.017629,0.010078,0.021898,0.961493
1,2070,1029743,0.0,U,Unknown,Unknown,50.0,61.0,1.0,0.0,...,-0.004586,0.016465,0.033132,0.010877,-0.015461,0.021982,0.017635,0.011317,0.016485,2.397848


**Обучение 2 уровня**

In [47]:
X_train = targets_lvl_2.drop('target', axis=1)
y_train = targets_lvl_2[['target']]

In [49]:
cat_feats = ['marital_status_code', 'homeowner_desc', 'hh_comp_desc', 'manufacturer', 'department', 'brand', 
             'commodity_desc', 'sub_commodity_desc', 'curr_size_of_product']

In [50]:
lgb = LGBMClassifier(objective='binary', max_depth=7, categorical_column=cat_feats)
lgb.fit(X_train, y_train)

[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves


LGBMClassifier(boosting_type='gbdt',
               categorical_column=['marital_status_code', 'homeowner_desc',
                                   'hh_comp_desc', 'manufacturer', 'department',
                                   'brand', 'commodity_desc',
                                   'sub_commodity_desc',
                                   'curr_size_of_product'],
               class_weight=None, colsample_bytree=1.0, importance_type='split',
               learning_rate=0.1, max_depth=7, min_child_samples=20,
               min_child_weight=0.001, min_split_gain=0.0, n_estimators=100,
               n_jobs=-1, num_leaves=31, objective='binary', random_state=None,
               reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1.0,
               subsample_for_bin=200000, subsample_freq=0)

In [51]:
targets_lvl_3 = data_val_lvl_2[['user_id', 'item_id']].copy()
targets_lvl_3.drop_duplicates(keep='first', inplace=True)

In [52]:
targets_lvl_3 = targets_lvl_3.merge(user_features_upd, on='user_id', how='left')
targets_lvl_3 = targets_lvl_3.merge(item_features_upd, on='item_id', how='left')
targets_lvl_3.shape

(77336, 63)

In [53]:
preds = lgb.predict(targets_lvl_3)
test_preds_proba = lgb.predict_proba(targets_lvl_3) #[:, 1]

In [54]:
targets_lvl_3['res'] = test_preds_proba[:, 1]
targets_lvl_3.head(2)

,user_id,item_id,marital_status_code,homeowner_desc,hh_comp_desc,age_desc_int,income_desc_int,household_size_desc_int,kid_category_desc_int,days_from_last_purchase,...,12_y,13_y,14_y,15_y,16_y,17_y,18_y,19_y,price,res
0,2449,721164,A,Homeowner,2 Adults Kids,40.0,187.0,5.0,3.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.99000,0.001522
1,314,820486,U,Homeowner,2 Adults Kids,50.0,87.0,3.0,1.0,27.0,...,0.005132,0.001326,0.004616,0.00486,-0.001289,0.010011,-0.002704,0.013835,2.09358,0.017706


In [55]:
targets_lvl_3.sort_values(['user_id', 'res'], ascending=False, inplace=True)
recs = targets_lvl_3.groupby('user_id')['item_id']

recomendations = []
for user, preds in recs:
    recomendations.append({'user_id': user, 'recomendations': preds.tolist()})

In [56]:
recomendations = pd.DataFrame(recomendations)

result_lvl_2 = data_val_lvl_2.groupby('user_id')['item_id'].unique().reset_index()
result_lvl_2.columns=['user_id', 'actual']

result_lvl_2 = result_lvl_2.merge(recomendations, on='user_id', how='left')
result_lvl_2.head(2)

,user_id,actual,recomendations
0,1,"[883616, 917704, 931860, 961554, 995242, 10020...","[1082185, 995242, 979707, 961554, 940947, 1005..."
1,6,"[909479, 7431990, 6553035, 12263667, 13382461,...","[995242, 840361, 849843, 5569230, 845208, 1075..."


**Отбор товаров**

In [57]:
top_valued_items = prices[(prices['price'] > 7)]
top_valued_items = top_valued_items.sort_values(by='price', ascending=False, na_position='last')
top_valued_items_list = top_valued_items['item_id'].tolist()

In [59]:
top_popular_items_m = prices[(prices['price'] > 1) & (prices['item_id'].isin(top_popular_items))]['item_id'].tolist() 
print(f'Популярных товаров до фильтрации - {len(top_popular_items)}, после - {len(top_popular_items_m)}')

Популярных товаров до фильтрации - 200, после - 176


In [60]:
top_valued_items_list = top_valued_items['item_id'].tolist()

In [61]:
result_lvl_2['rec'] = result_lvl_2['user_id'].progress_apply\
        (lambda x: get_recommendation_5(x, result_lvl_2, item_features, top_popular_items_m, top_valued_items_list)[0])

100%|██████████████████████████████████████████████████████████████████████████████| 1899/1899 [04:14<00:00,  7.45it/s]


In [62]:
result_lvl_2.head(2)

,user_id,actual,recomendations,rec
0,1,"[883616, 917704, 931860, 961554, 995242, 10020...","[1082185, 995242, 979707, 961554, 940947, 1005...","[1321795, 823704, 824005, 1082185, 995242]"
1,6,"[909479, 7431990, 6553035, 12263667, 13382461,...","[995242, 840361, 849843, 5569230, 845208, 1075...","[1329768, 823704, 824005, 995242, 840361]"


**Целевая метрика**

In [63]:
result_lvl_2.progress_apply(lambda row: money_precision_at_k(row['rec'], row['actual'], prices), axis=1).mean()

100%|█████████████████████████████████████████████████████████████████████████████| 1899/1899 [00:14<00:00, 135.27it/s]


0.5065662737981665

In [64]:
test_data_upd = test_data[['user_id', 'item_id']].copy()
test_data_upd.drop_duplicates(keep='first', inplace=True)

In [65]:
test_data_upd.shape

(79282, 2)

In [66]:
test_data_upd = test_data_upd.merge(user_features_upd, on='user_id', how='left')
test_data_upd = test_data_upd.merge(item_features_upd, on='item_id', how='left')
test_data_upd.shape

(79282, 63)

In [67]:
test_preds_proba_2 = lgb.predict_proba(test_data_upd)[:, 1]

In [68]:
test_data_upd['proba'] = test_preds_proba_2
test_data_upd = test_data_upd[test_data_upd['price'] > 1]

In [69]:
result = test_data_upd.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']

In [70]:
test_data_upd.sort_values(['user_id', 'proba'], ascending=False, inplace=True)
recs = test_data_upd.groupby('user_id')['item_id']

recomendations = []
for user, preds in recs:
    recomendations.append({'user_id': user, 'recomendations': preds.tolist()})

In [71]:
recomendations = pd.DataFrame(recomendations)

In [72]:
result = result.merge(recomendations, on='user_id', how='left')
result.head(2)

,user_id,actual,recomendations
0,1,"[880007, 883616, 931136, 938004, 940947, 95292...","[962568, 979707, 961554, 1004906, 940947, 5582..."
1,2,"[820291, 826784, 826835, 829009, 866211, 87060...","[1133018, 899624, 951590, 1053690, 866211, 102..."


In [73]:
result['rec'] = result['user_id'].progress_apply\
        (lambda x: get_recommendation_5(x, result, item_features, top_popular_items_m, top_valued_items_list)[0])

100%|██████████████████████████████████████████████████████████████████████████████| 1865/1865 [03:32<00:00,  8.79it/s]


In [74]:
result.head(2)

,user_id,actual,recomendations,rec
0,1,"[880007, 883616, 931136, 938004, 940947, 95292...","[962568, 979707, 961554, 1004906, 940947, 5582...","[7410161, 823704, 824005, 962568, 979707]"
1,2,"[820291, 826784, 826835, 829009, 866211, 87060...","[1133018, 899624, 951590, 1053690, 866211, 102...","[7410217, 823704, 824005, 1133018, 899624]"


In [78]:
result['pres'] = result.progress_apply(lambda row: \
                                                money_precision_at_k(row['rec'], row['actual'], prices), axis=1)#.mean()

100%|█████████████████████████████████████████████████████████████████████████████| 1865/1865 [00:14<00:00, 125.93it/s]


In [79]:
result['pres'].mean()

0.5255157784417759

Значение допустимо

In [80]:
result['rec_txt'] = result['rec'].progress_apply(lambda x: str(x)[1: -1])

100%|███████████████████████████████████████████████████████████████████████████| 1865/1865 [00:00<00:00, 64349.40it/s]


In [81]:
result.head(2)

,user_id,actual,recomendations,rec,pres,rec_txt
0,1,"[880007, 883616, 931136, 938004, 940947, 95292...","[962568, 979707, 961554, 1004906, 940947, 5582...","[7410161, 823704, 824005, 962568, 979707]",0.108961,"7410161, 823704, 824005, 962568, 979707"
1,2,"[820291, 826784, 826835, 829009, 866211, 87060...","[1133018, 899624, 951590, 1053690, 866211, 102...","[7410217, 823704, 824005, 1133018, 899624]",0.777916,"7410217, 823704, 824005, 1133018, 899624"


In [82]:
final_path = path + 'recommendation.csv'
result.to_csv(final_path, columns=['user_id', 'rec_txt'], header=['user_id', 'recommended_items_id'], index=False)